<a href="https://colab.research.google.com/github/nhatchung14/PythonNotebooks/blob/master/3_Exercise_Iris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [268]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive
%cd 'My Drive'/'Colab Notebooks'/Lab6/2-Decision-Tree


Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive
/gdrive/My Drive/Colab Notebooks/Lab6/2-Decision-Tree


# Bài tập Decision Tree & KNN w/ Iris.csv

In [0]:
# Data
import pandas as pd

# Model
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier  
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

# Timer
import time

In [0]:
dataset = pd.read_csv('Iris.csv')

Data info

In [271]:
# get how many instances (rows) and how many attributes (columns)
dataset.shape

(150, 6)

In [272]:
#show BASIC INFORMATION: max, min, mean của các columns trong dataset
dataset.describe()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,75.500000,5.843333,3.054000,3.758667,1.198667
std,43.445368,0.828066,0.433594,1.764420,0.763161
min,1.000000,4.300000,2.000000,1.000000,0.100000
25%,38.250000,5.100000,2.800000,1.600000,0.300000
50%,75.500000,5.800000,3.000000,4.350000,1.300000
75%,112.750000,6.400000,3.300000,5.100000,1.800000
max,150.000000,7.900000,4.400000,6.900000,2.500000


In [273]:
# T number of instances (rows) that belong to each class. 
dataset.groupby('Species').size()                 # diabetes.csv

Species
Iris-setosa        50
Iris-versicolor    50
Iris-virginica     50
dtype: int64

Preprocess

In [0]:
# Determine features and labels
feature_columns = ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm','PetalWidthCm']
X = dataset[feature_columns].values
y = dataset['Species'].values

In [0]:
# Labels are categorical variables. Therefore, we have to transform them into a numeric format as KNeighborsClassifier does not accept string labels

# Iris-setosa correspond to 0
# Iris-versicolor correspond to 1
# Iris-virginica correspond to 2
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
yl = le.fit_transform(y)

In [276]:
# Spliting dataset into training set and test set
# training set: to build classifier
# test set: to evaluate classifier  
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, yl, test_size = 0.2, random_state = 0)
print(X_train.shape) # 80% rows dùng để train
print(y_train.shape)
print(X_test.shape) # 20% rows dùng để test
print(y_test.shape)

(120, 4)
(120,)
(30, 4)
(30,)


## Use Decision Tree to classify the Iris dataset
- dataset: 'Iris.csv'


In [277]:
# model (k = 1)
classifier = KNeighborsClassifier(n_neighbors=1)

# Fitting the model
classifier.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform')

In [278]:
# Predicting on the test set
start = time.time()
y_pred = classifier.predict(X_test)
print("Elapsed time = ", time.time() - start)
y_pred

Elapsed time =  0.003981828689575195


array([2, 1, 0, 2, 0, 2, 0, 1, 1, 1, 2, 1, 1, 1, 1, 0, 1, 1, 0, 0, 2, 1,
       0, 0, 2, 0, 0, 1, 1, 0])

In [279]:
cm = confusion_matrix(y_test, y_pred)
cm

array([[11,  0,  0],
       [ 0, 13,  0],
       [ 0,  0,  6]])

In [280]:
#Sử dụng hàm accuracy_score
accuracy = accuracy_score(y_test, y_pred)*100
print('Accuracy of our model is equal ' + str(round(accuracy, 2)) + ' %.')

Accuracy of our model is equal 100.0 %.


In [281]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        11
           1       1.00      1.00      1.00        13
           2       1.00      1.00      1.00         6

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



In [282]:
cv = StratifiedKFold(n_splits=10)            # Desired number of Cross Validation folds
fold_accuracy = list()

for train_fold, valid_fold in cv.split(X, y):
    f_train = dataset.loc[train_fold] # Extract train data with cv indices
    f_valid = dataset.loc[valid_fold] # Extract valid data with cv indices

    model = classifier.fit(X = f_train.drop(['Species'], axis=1), 
                            y = f_train["Species"]) # We fit the model with the fold train data
    valid_acc = classifier.score(X = f_valid.drop(['Species'], axis=1), 
                            y = f_valid["Species"])# We calculate accuracy with the fold validation data
    fold_accuracy.append(valid_acc)

avg = sum(fold_accuracy)/len(fold_accuracy)
print(avg)

0.9600000000000002


## Use KNN to classify the Iris dataset
- dataset: 'Iris.csv'


In [283]:
# model
classifier = DecisionTreeClassifier(criterion='entropy')  

# Fitting the model
classifier.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [284]:
# Predicting on the test set
start = time.time()
y_pred = classifier.predict(X_test)
print("Elapsed time = ", time.time() - start)
y_pred

Elapsed time =  0.0003662109375


array([2, 1, 0, 2, 0, 2, 0, 1, 1, 1, 2, 1, 1, 1, 1, 0, 1, 1, 0, 0, 2, 1,
       0, 0, 2, 0, 0, 1, 1, 0])

In [285]:
cm = confusion_matrix(y_test, y_pred)
cm

array([[11,  0,  0],
       [ 0, 13,  0],
       [ 0,  0,  6]])

In [286]:
#Sử dụng hàm accuracy_score
accuracy = accuracy_score(y_test, y_pred)*100
print('Accuracy of our model is equal ' + str(round(accuracy, 2)) + ' %.')

Accuracy of our model is equal 100.0 %.


In [287]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        11
           1       1.00      1.00      1.00        13
           2       1.00      1.00      1.00         6

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



In [288]:
cv = StratifiedKFold(n_splits=10)            # Desired number of Cross Validation folds
fold_accuracy = list()

for train_fold, valid_fold in cv.split(X, y):
    f_train = dataset.loc[train_fold] # Extract train data with cv indices
    f_valid = dataset.loc[valid_fold] # Extract valid data with cv indices

    model = classifier.fit(X = f_train.drop(['Species'], axis=1), 
                            y = f_train["Species"]) # We fit the model with the fold train data
    valid_acc = classifier.score(X = f_valid.drop(['Species'], axis=1), 
                            y = f_valid["Species"])# We calculate accuracy with the fold validation data
    fold_accuracy.append(valid_acc)

avg = sum(fold_accuracy)/len(fold_accuracy)
print(avg)

0.9533333333333334
